# Property Value Trends in NH
## Combining and Cleaning Data

In [2]:
import pandas as pd
import re

## Read and Clean 2011 Data

Property tax data sourced from the [NH Dept. of Revenue Administration](https://www.revenue.nh.gov/mun-prop/municipal/documents/2009-local.pdf). I exported the PDF to a csv file. Some cleanup of the file was necessary.

In [140]:
df_2011 = pd.read_excel('../Raw Data/2011-local.xlsx', engine="openpyxl")
df_2011 = df_2011.drop(['P', 'Date\nIssued', 'Unnamed: 10'], axis=1)
df_2011.columns = ['Town', 'Valuation', 'Town Tax', 'Local Ed. Tax', 'State Ed. Tax', 'County Tax', 'Total Tax', 'Total Commitment']

#Revise town name formatting to match 2020 data
df_2011['Town'] = df_2011['Town'].str.title()
df_2011['Town'] = df_2011['Town'].str.replace("'S", "'s")

df_2011 = df_2011.set_index('Town')

#Rectify a few towns who's names are different between the 2011 and 2020 datasets
df_2011 = df_2011.rename(index={'Thom. & Mes.':'Thom. & Mes. Purchase',
                                'Low & Burbank Gr': "Low & Burbank's Grant",
                                'Atk. & Gilmanton': 'Atkinson & Gilmanton Academy Grant'})

df_2011.head()

,Valuation,Town Tax,Local Ed. Tax,State Ed. Tax,County Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,94713062,6.83,8.65,2.51,3.20,21.19,1996801
Albany,103641581,3.05,4.15,2.50,1.06,10.76,1089760
Alexandria,200605414,5.57,12.70,2.60,1.50,22.37,4403307
Allenstown,267129444,7.93,16.30,2.34,2.73,29.30,7680063
Alstead,178908827,4.91,10.74,2.38,2.91,20.94,3734318


### Adjust valuation for inflation. 
`$1.00 (2011) = $1.17 (2020)`, according the [U.S. Bureau of Labor Statistics](https://data.bls.gov/cgi-bin/cpicalc.pl). First, we need to clean up the valuation data.

In [121]:
def clean_int(x):
    
    x = str(x)
    x = x.replace(',','')
    x = x.replace('*','')
    x = x.replace('$','')
    x = x.strip()
    x = re.search('([0-9]*)', x)[0]        
   
    try:
        return int(x)
    except:
        return -999

In [141]:
df_2011.Valuation = df_2011.Valuation.apply(clean_int).astype('int')
df_2011.Valuation = df_2011.Valuation * 1.17
df_2011['Total Commitment'] = df_2011['Valuation'] * df_2011['Total Tax']

df_2011[['Valuation', 'Total Commitment']] = df_2011[['Valuation', 'Total Commitment']].astype('int')

In [142]:
df_2011

,Valuation,Town Tax,Local Ed. Tax,State Ed. Tax,County Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,110814282,6.83,8.65,2.51,3.20,21.19,2348154647
Albany,121260649,3.05,4.15,2.50,1.06,10.76,1304764591
Alexandria,234708334,5.57,12.70,2.60,1.50,22.37,5250425440
Allenstown,312541449,7.93,16.30,2.34,2.73,29.30,9157464469
Alstead,209323327,4.91,10.74,2.38,2.91,20.94,4383230479
...,...,...,...,...,...,...,...
Winchester,330630545,7.18,17.50,2.11,3.24,30.03,9928835287
Windham,2369279682,3.79,15.77,2.46,1.06,23.08,54682975081
Windsor,26781515,0.91,8.81,3.57,1.19,14.48,387796341


## Read and clean 2020 data

In [133]:
usecols=['Municipality',
         'Valuation w/ Utils',
         'Municipal',
         'County',
         'State Ed.',
         'Local Ed.', 
         'Total Rate', 
         'Total Commitment']

df_2020 = pd.read_excel('../Raw Data/20-tax-rates.xlsx', 
                        engine='openpyxl',
                        usecols=usecols,
                        skiprows=5)

# Match w/2011 dataframe
df_2020.columns = ['Town',
                   'Valuation',
                   'Town Tax',
                   'County Tax',
                   'State Ed. Tax',
                   'Local Ed. Tax',
                   'Total Tax', 
                   'Total Commitment']

#Edit names for unincorporate places to match 2011 data
df_2020['Town'] = df_2020['Town'].str.replace(' \(U\)', '', regex=True)

df_2020 = df_2020.set_index('Town')

df_2020

,Valuation,Town Tax,County Tax,State Ed. Tax,Local Ed. Tax,Total Tax,Total Commitment
Town,,,,,,,
Acworth,100103637,9.66,2.60,1.97,15.14,29.37,2921681
Albany,130141621,2.20,1.08,1.71,7.06,12.05,1546965
Alexandria,198181477,9.32,1.99,2.20,14.41,27.92,5460145
Allenstown,299434087,9.62,2.76,2.03,14.89,29.30,8652469
Alstead,193481509,5.51,3.46,1.92,13.53,24.42,4692133
...,...,...,...,...,...,...,...
Winchester,333847858,7.09,3.15,1.68,16.85,28.77,9436256
Windham,3059617070,2.99,0.85,1.84,13.45,19.13,58209101
Windsor,28777018,0.42,0.94,2.03,6.42,9.81,276423


## Combine dataframes

In [143]:
df = df_2011.merge(df_2020, left_index=True, right_index=True, suffixes=(' 2011', ' 2020'), how='outer')
df

,Valuation 2011,Town Tax 2011,Local Ed. Tax 2011,State Ed. Tax 2011,County Tax 2011,Total Tax 2011,Total Commitment 2011,Valuation 2020,Town Tax 2020,County Tax 2020,State Ed. Tax 2020,Local Ed. Tax 2020,Total Tax 2020,Total Commitment 2020
Town,,,,,,,,,,,,,,
Acworth,110814282,6.83,8.65,2.51,3.20,21.19,2348154647,100103637,9.66,2.60,1.97,15.14,29.37,2921681
Albany,121260649,3.05,4.15,2.50,1.06,10.76,1304764591,130141621,2.20,1.08,1.71,7.06,12.05,1546965
Alexandria,234708334,5.57,12.70,2.60,1.50,22.37,5250425440,198181477,9.32,1.99,2.20,14.41,27.92,5460145
Allenstown,312541449,7.93,16.30,2.34,2.73,29.30,9157464469,299434087,9.62,2.76,2.03,14.89,29.30,8652469
Alstead,209323327,4.91,10.74,2.38,2.91,20.94,4383230479,193481509,5.51,3.46,1.92,13.53,24.42,4692133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Winchester,330630545,7.18,17.50,2.11,3.24,30.03,9928835287,333847858,7.09,3.15,1.68,16.85,28.77,9436256
Windham,2369279682,3.79,15.77,2.46,1.06,23.08,54682975081,3059617070,2.99,0.85,1.84,13.45,19.13,58209101
Windsor,26781515,0.91,8.81,3.57,1.19,14.48,387796341,28777018,0.42,0.94,2.03,6.42,9.81,276423


## Export to a CSV

In [144]:
df.to_csv('../Cleaned Data/NH_Property_Tax_Rates_2011-20.csv')